In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps

#import crud module
from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model

username = "aacuser"
password = "password"
shelter = AnimalShelter()



df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'Grazioso_Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
  
    html.Center(html.B(html.H1('Shawn Frye Dashboard'))),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    
    html.H1('Shawn Frye'), #unique identifier
    html.Hr(),
    html.Div([
        dcc.Dropdown(  #dropdown box for different breed categories
            id='demo-dropdown',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain or Wilderness', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='Water'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,       
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    html.Br(),
    html.Hr(),    
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="pie-chart", #pie chart display
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',  #map display
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

       
#get data about breed categories
@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    if (value =='water'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Pitbull Mix",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'mountain'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.read(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')
    


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#generate pie chart
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(
        dff,
        names='breed',
    )
    return fig

#generate map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, derived_virtual_selected_rows):
    
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    animalChoice = None
    if not derived_virtual_selected_rows:
        animalChoice = dff.iloc[0]
    else:
        animalChoice = dff.iloc[derived_virtual_selected_rows[0]]
        
    latitude = animalChoice[12]
    longitude = animalChoice[13]
    breed = animalChoice[3]
    name = animalChoice[8]
    
  
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            dl.Marker(position=[latitude, longitude], children=[               
                
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),                  
                    html.P(name)
                ])
            ])
        ])
    ]

#run
app
